In [11]:
!pip install requests
!pip install fhir.resources
!pip install FastAPI
!pip install uvicorn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from IPython.display import display, HTML

In [4]:
import json
file_env= "fast-api-PKM-DEV-env.json"
with open (file_env) as json_file:
    env_dict = json.load(json_file)
auth_url = env_dict["auth_url"]
base_url = env_dict["base_url"]
consent_url = env_dict["consent_url"]
client_id = env_dict["client_id"]
client_secret = env_dict["client_secret"]
Org_id= env_dict["Org_id"]
print(env_dict)

{'auth_url': 'https://api-satusehat-dev.dto.kemkes.go.id/oauth2/v1', 'base_url': 'https://api-satusehat-dev.dto.kemkes.go.id/fhir-r4/v1', 'consent_url': 'https://api-satusehat-dev.dto.kemkes.go.id/consent/v1', 'client_id': 'xlQvaLCrGmns8VqM3jm2sxauhFipSqD8DfC9wZ9YxL5lRpyU', 'client_secret': 'g6xiprB2abrD2hN3x9wVmp7ESN36ZRDPcfkoNklWFNvCGfaOCppYz6qyJJ0sOTsm', 'Org_id': '10000005'}


In [112]:
import requests
import time
from datetime import datetime, timedelta

def get_token():
    # 1.Data to be sent in body as application/x-www-form-urlencoded
    data = f"client_id={client_id}&client_secret={client_secret}"

    # 2. Send the HTTP Request
    url = f"{auth_url}/accesstoken?grant_type=client_credentials"  # Change this URL to your server's endpoint
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    # Measure response time
    start_time = datetime.now()
    response = requests.request("POST", url, data=data, headers=headers)
    end_time = datetime.now()

    # 3. Log the Response
    response_time = (end_time - start_time)
    response_code = response.status_code
    response_body = response.json()  # Assumes the response is in JSON format

    print(f"Response Time: {response_time}")
    print(f"Response Code: {response_code}")
    print("Response Body:")
    print(response.text)

    # Check content type
    print("Content-Type:", response.headers.get('Content-Type'))
    try:
        data = response.json()  # Attempt to get JSON data
        print("Token:", data.get('access_token'))
    except ValueError:
        print("Failed to parse JSON:", response.text)

    data= response.json()
    bearer_token= data.get('access_token')
    expiry_time= start_time + timedelta(seconds = int(data.get('expires_in')))
    expiry_time= expiry_time.strftime("%Y-%m-%d %H:%M:%S")
    print(expiry_time)
    return bearer_token, expiry_time

def post_resource(resource, bearer_token):
    # 1.Data to be sent in body as application/x-www-form-urlencoded 
    try:
        data = json.loads(resource)
        payload= resource
        resource_type= data.get("resourceType")
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        return

    # 2. Send the HTTP Request
    url= f"{base_url}/{resource_type}"
    headers= {"Content-Type": "application/json",
              "Authorization": f"Bearer {bearer_token}"}

    # Measure response time
    start_time = datetime.now()
    response = requests.request("POST", url, data=payload, headers=headers)
    end_time = datetime.now()

    # 3. Log the Response
    response_time = (end_time - start_time)
    response_code = response.status_code
    response_body = response.text  # Assumes the response is in JSON format

    print(f"Response Time: {response_time}")
    print(f"Response Code: {response_code}")
    print("Response Body:")
    print(response.text)

    # Check content type
    print("Content-Type:", response.headers.get('Content-Type'))
    try:
        data = response.json()  # Attempt to get JSON data
        print("Uuid:", data.get('id'))
    except ValueError:
        print("Failed to parse JSON:", response.text)

    data= response.json()
    return response_body



In [97]:
access_token= get_token()

Response Time: 0:00:00.737717
Response Code: 200
Response Body:
{
  "refresh_token_expires_in": "0",
  "api_product_list": "[api-sandbox, api-satusehat-dev]",
  "api_product_list_json": [
    "api-sandbox",
    "api-satusehat-dev"
  ],
  "organization_name": "ihs-prod-1",
  "developer.email": "ihs.developer@dto.kemkes.go.id",
  "token_type": "BearerToken",
  "issued_at": "1713173564072",
  "client_id": "xlQvaLCrGmns8VqM3jm2sxauhFipSqD8DfC9wZ9YxL5lRpyU",
  "access_token": "oGDA3Pd9AK0xqbGpbLj0bYqXB2sx",
  "application_name": "5b77460b-bb57-4609-a5cb-54a1b49c8632",
  "scope": "",
  "expires_in": "3599",
  "refresh_count": "0",
  "status": "approved"
}
Content-Type: application/json
Token: oGDA3Pd9AK0xqbGpbLj0bYqXB2sx
2024-04-15 17:32:43


In [98]:
bearer_token= access_token[0]
expiry_time = datetime.strptime(access_token[1],"%Y-%m-%d %H:%M:%S")
print(bearer_token)
print(expiry_time)

oGDA3Pd9AK0xqbGpbLj0bYqXB2sx
2024-04-15 17:32:43


In [113]:
now= datetime.now()
if now < (expiry_time - timedelta(seconds= 100)):
    print("yes")
else: 
    access_token= get_token()
    bearer_token= access_token[0]
    expiry_time = datetime.strptime(access_token[1],"%Y-%m-%d %H:%M:%S")

yes


In [115]:
from random import randint
from fhir.resources.R4B.encounter import Encounter
from fhir.resources.R4B.identifier import Identifier
from fhir.resources.R4B.reference import Reference
from fhir.resources.R4B.codeableconcept import CodeableConcept
from fhir.resources.R4B.coding import Coding
from fhir.resources.R4B.period import Period
from fhir.resources.R4B.encounter import EncounterParticipant
from fhir.resources.R4B.encounter import EncounterStatusHistory 
from fhir.resources.R4B.encounter import EncounterLocation
from fhir.resources.R4B.encounter import EncounterDiagnosis
from datetime import datetime,timedelta,timezone

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

x = 1

EncounterClass_Dict= {
    "Ambulatory": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "AMB",
        "display": "ambulatory"
    },
    "Emergency": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "EMER",
        "display": "emergency"
    },
    "Field": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "FLD",
        "display": "field"
    },
    "Home Health": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "HH",
        "display": "home health"
    },
    "Inpatient Encounter": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "IMP",
        "display": "inpatient encounter"
    },
    "Inpatient Acute": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "ACUTE",
        "display": "inpatient acute"
    },
    "Inpatient Non-Acute": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "NONAC",
        "display": "inpatient non-acute"
    },
    "Observation Encounter": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "OBSENC",
        "display": "observation encounter"
    },
    "Pre-Admission": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "PRENC",
        "display": "pre-admission"
    },
    "Short Stay": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "SS",
        "display": "short stay"
    },
    "Virtual": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "VR",
        "display": "virtual"
    }
}

ParticipantType_Dict= {
    "Admitter": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "ADM",
        "display": "admitter"
    },
    "Attender": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "ATND",
        "display": "attender"
    },
    "Callback Contact": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "CALLBCK",
        "display": "callback contact"
    },
    "Consultant": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "CON",
        "display": "consultant"
    },
    "Discharger": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "DIS",
        "display": "discharger"
    },
    "Escort": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "ESC",
        "display": "escort"
    },
    "Referrer": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "REF",
        "display": "referrer"
    },
    "Secondary Performer": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "SPRF",
        "display": "secondary performer"
    },
    "Primary Performer": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "PPRF",
        "display": "primary performer"
    },
    "Participation": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ParticipationType",
        "code": "PART",
        "display": "Participation"
    },
    "Translator": {
        "system": "http://terminology.hl7.org/CodeSystem/participant-type",
        "code": "translator",
        "display": "Translator"
    },
    "Emergency": {
        "system": "http://terminology.hl7.org/CodeSystem/participant-type",
        "code": "emergency",
        "display": "Emergency"
    }
}

#for x in df_conditions:
#    EncounterDiagnosis()

class Encounter_IDFHIR:
    def __init__(self):
        participant_individual = Reference(reference= f"Practitioner/{Practitioner_Id}", display= f"{Practitioner_String}")
        Encounter_location = [EncounterLocation(location= Reference(reference= f"Location/{Location_Id}", display= f"{Location_String}"))]
        Encounter_serviceProvider = Reference(reference='Organization/' + Org_Id)
        Encounter_MRID = "E" + DateTime.strftime('%Y%m%d') + str(random_with_N_digits(4))
        participant_type = [CodeableConcept(coding=[Coding(
            system= ParticipantType_Dict[Practitioner_Type]["system"],
            code= ParticipantType_Dict[Practitioner_Type]["code"],
            display= ParticipantType_Dict[Practitioner_Type]["display"])])
        ]

        class_type= Coding(
            system= EncounterClass_Dict[Encounter_Class]["system"],
            code= EncounterClass_Dict[Encounter_Class]["code"],
            display= EncounterClass_Dict[Encounter_Class]["display"]
        )

        if Encounter_Status== "arrived":
            Encounter_period = Period(start= f"{DateTime_Arrival}")
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}")
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period)]
            Encounter_Condition= None

        if Encounter_Status== "in-progress":
            Encounter_period = Period(start= f"{DateTime_Arrival}")
            DateTime_InProgress= DateTime + timedelta(minutes= 15)
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}", end= f"{DateTime_InProgress}")
            Encounter_inprogressHistory_period = Period(start= f"{DateTime_Arrival}")
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period),
                                EncounterStatusHistory(status="in-progress", period= Encounter_inprogressHistory_period)]
            Encounter_Condition= None

        if Encounter_Status== "finished":
            Encounter_period = Period(start= f"{DateTime_Arrival}", end= f'{DateTime_Finished}')
            DateTime_Finished= DateTime + timedelta(minutes= 15)
            Encounter_arrivedHistory_period = Period(start= f"{DateTime_Arrival}", end= f"{DateTime_InProgress}")
            Encounter_inprogressHistory_period = Period(start= f"{DateTime_Arrival}", end= f'{DateTime_Finished}')
            Encounter_finishedHistory_period= Period(start= f'{DateTime_Finished}', end= f'{DateTime_Finished}')
            Encounter_statusHistory= [EncounterStatusHistory(status= "arrived", period= Encounter_arrivedHistory_period),
                                EncounterStatusHistory(status="in-progress", period= Encounter_inprogressHistory_period),
                                EncounterStatusHistory(status="finished", period= Encounter_finishedHistory_period)]
        
        # Create EncounterParticipant instance
        Encounter_participant = EncounterParticipant(
            type=participant_type,
            individual=participant_individual
        )

        self.encounter= Encounter(
            status= Encounter_Status,
            period= Encounter_period,
            identifier= [(Identifier(system= f'sys-ids.kemkes.go.id/Encounter/{Org_Id}', value= f"{Encounter_MRID}"))],
            class_fhir= class_type,
            subject= Reference(reference= f"Patient/{Patient_Id}", display= f'{Patient_String}'),
            participant= [Encounter_participant],
            serviceProvider= Encounter_serviceProvider,
            location= Encounter_location,
            diagnosis= Encounter_Condition,
            statusHistory= Encounter_statusHistory
        )
        return None

    def __str__(self):
        # Return a string representation of the Encounter
        # Assuming that the fhir.resources Encounter class has a method to convert to a dict
        return str(self.encounter.json(indent= 4))

    def to_JSON(self): 
        return self.encounter.json(indent= 4)

# Insert used parameters
Encounter_uuid = "{{Encounter_uuid}}" # @param {type : "string"}
Org_Id = Org_id # @param {type : "string"}
MR_Gen= random_with_N_digits(4)
Encounter_MRID= '202403A000' + str(MR_Gen)
Encounter_Class= "Ambulatory"
Encounter_Status= "in-progress"
identifier_value= f"{Encounter_MRID}"
Patient_Id = "100000030015" # @param {type : "string"}
Patient_PMID= "100000030015" # @param {type : "string"}
Patient_String = "Diana Smith" # @param {type : "string"}
Practitioner_Id = "N10000001" # @param {type : "string"}
Practitioner_Type= "Attender"
Practitioner_String = "dr. Bronsig" # @param {type : "string"}
Location_Id ="29bc6779-e877-4cb4-8f71-5db4d0cdda0a" # @param {type : "string"}
Location_String = "Ruang Poli SMF Rajal" # @param {type : "string"}
DateTime_Year = 2023
DateTime_Month= 9
DateTime_Day= 2
DateTime_Hour= 0
DateTime_Min= 0

DateTime= datetime(
    year= DateTime_Year, 
    month= DateTime_Month, 
    day= DateTime_Day, 
    hour= DateTime_Hour, 
    minute= DateTime_Min,
    tzinfo= timezone.utc
    )

DateTime_Arrival= DateTime

resource= Encounter_IDFHIR()
resource= resource.to_JSON()
print(resource)

{
    "resourceType": "Encounter",
    "identifier": [
        {
            "system": "sys-ids.kemkes.go.id/Encounter/10000005",
            "value": "E202309029090"
        }
    ],
    "status": "in-progress",
    "statusHistory": [
        {
            "status": "arrived",
            "period": {
                "start": "2023-09-02T00:00:00+00:00",
                "end": "2023-09-02T00:15:00+00:00"
            }
        },
        {
            "status": "in-progress",
            "period": {
                "start": "2023-09-02T00:00:00+00:00"
            }
        }
    ],
    "class": {
        "system": "http://terminology.hl7.org/CodeSystem/v3-ActCode",
        "code": "AMB",
        "display": "ambulatory"
    },
    "subject": {
        "reference": "Patient/100000030015",
        "display": "Diana Smith"
    },
    "participant": [
        {
            "type": [
                {
                    "coding": [
                        {
                            "syste

In [116]:
Encounter_response= post_resource(resource, bearer_token)

Response Time: 0:00:01.592400
Response Code: 201
Response Body:
{"class":{"code":"AMB","display":"ambulatory","system":"http://terminology.hl7.org/CodeSystem/v3-ActCode"},"id":"d55b88ab-1feb-4daf-840a-ba2c2679e628","identifier":[{"system":"sys-ids.kemkes.go.id/Encounter/10000005","value":"E202309029090"}],"location":[{"location":{"display":"Ruang Poli SMF Rajal","reference":"Location/29bc6779-e877-4cb4-8f71-5db4d0cdda0a"}}],"meta":{"lastUpdated":"2024-04-15T09:37:39.612770+00:00","versionId":"MTcxMzE3Mzg1OTYxMjc3MDAwMA"},"participant":[{"individual":{"display":"dr. Bronsig","reference":"Practitioner/N10000001"},"type":[{"coding":[{"code":"ATND","display":"attender","system":"http://terminology.hl7.org/CodeSystem/v3-ParticipationType"}]}]}],"period":{"start":"2023-09-02T00:00:00+00:00"},"resourceType":"Encounter","serviceProvider":{"reference":"Organization/10000005"},"status":"in-progress","statusHistory":[{"period":{"end":"2023-09-02T00:15:00+00:00","start":"2023-09-02T00:00:00+00:00"

In [108]:
Encounter_response

'{"class":{"code":"AMB","display":"ambulatory","system":"http://terminology.hl7.org/CodeSystem/v3-ActCode"},"id":"3b091f63-61f2-4e79-b033-ca2e3142e903","identifier":[{"system":"sys-ids.kemkes.go.id/Encounter/10000005","value":"E202309026430"}],"location":[{"location":{"display":"Ruang Poli SMF Rajal","reference":"Location/29bc6779-e877-4cb4-8f71-5db4d0cdda0a"}}],"meta":{"lastUpdated":"2024-04-15T09:33:25.450430+00:00","versionId":"MTcxMzE3MzYwNTQ1MDQzMDAwMA"},"participant":[{"individual":{"display":"dr. Bronsig","reference":"Practitioner/N10000001"},"type":[{"coding":[{"code":"ATND","display":"attender","system":"http://terminology.hl7.org/CodeSystem/v3-ParticipationType"}]}]}],"period":{"start":"2023-09-02T00:00:00+00:00"},"resourceType":"Encounter","serviceProvider":{"reference":"Organization/10000005"},"status":"in-progress","statusHistory":[{"period":{"end":"2023-09-02T00:15:00+00:00","start":"2023-09-02T00:00:00+00:00"},"status":"arrived"},{"period":{"start":"2023-09-02T00:00:00+0

In [107]:
json_string= json.load(Encounter_post, indent=4)
json_string

AttributeError: 'str' object has no attribute 'read'